In [1]:
from imports import * # Catchall for commonly used imports. Will still throw linter warnings for imports not explicitly called below

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from bs4 import BeautifulSoup
import requests
import os
import re


import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

### Acquire and process Central Alberta cattle auction data from three sites:
- vjvauction.com (Ponoka, Rimbey, and Westlock markets)
- oldsauction.com
- drylandcattle.com (Veteran market)

In [2]:
# Establish urls

vjv_ponoka_url = 'https://vjvauction.com/ponoka/market-report/cattle'
vjv_rimbey_url = 'https://vjvauction.com/rimbey/market-report/cattle'
vjv_westlock_url = 'https://vjvauction.com/westlock/market-report/cattle'
olds_url = 'https://www.oldsauction.com/market-report'
dryland_url = 'http://www.drylandcattle.com/marketreports.aspx'



In [3]:
# Test site response
response = requests.get(vjv_ponoka_url)
print(response.status_code)
print('--------')
print(response.text)

200
--------


<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml">
<head><meta charset="utf-8" /><title>
	Cattle Market Report | VJV Ponoka
</title>
    <script src="/js/jquery-2.1.1.min.js" type="text/javascript"></script>

    <script src="/js/func.js" type="text/javascript"></script>
    <script src="/js/jquery.easing-1.3.min.js" type="text/javascript"></script>
    <script src="/js/jquery.cookie.js" type="text/javascript"></script>

    <link href="../../css/styles.min.css?v=2.1" rel="stylesheet" /><link href="../../css/print.css?v=1.2" media="print" rel="stylesheet" /><link href="../../favicon.png?v=1" rel="shortcut icon" type="image/x-icon" />

    <!-- start: Mobile Specific -->
    <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1" />
    <!-- end: Mobile Specific -->

    <script type="text/javascript">
        $(function () {
            var selector = '#catnav';
            $(selector).on('click', function () {
                $.cook

In [4]:
# Make soup object to parse html
soup = BeautifulSoup(response.content, 'html.parser')
# Parse the html
soup.select('table', class_= 'table_marketcurrent')


[<table class="table_marketcurrent">
 <tr>
 <th class="white txt-left"><i class="fa fa-history white mR-sm"></i>Click in table to get category history.</th>
 <th colspan="2">Last Auction</th>
 <th colspan="2">Previous Auction 1</th>
 <th colspan="2">Previous Auction 2</th>
 </tr>
 <tr>
 <th class="txt-left">Week Ending: </th>
 <th colspan="2">
 <span id="plh_MainContent_lblWeekC">Wed Aug 31, 2022</span>
 </th>
 <th colspan="2">
 <span id="plh_MainContent_lblWeekL">Wed Aug 24, 2022</span>
 </th>
 <th colspan="2">
 <span id="plh_MainContent_lblWeekY">Wed Aug 17, 2022</span>
 </th>
 </tr>
 <tr class="ponoka_back_lighter">
 <td class="txt-left">Head Sold: </td>
 <td class="txt-center" colspan="2">
 <span id="plh_MainContent_lblHead_C">3529</span>
 </td>
 <td class="txt-center" colspan="2">
 <span id="plh_MainContent_lblHead_L">3114</span>
 </td>
 <td class="txt-center" colspan="2">
 <span id="plh_MainContent_lblHead_Y">822</span>
 </td>
 </tr>
 <tr>
 <td class="ponoka_back" colspan="7">Sla

In [5]:
[span.text for span in soup.find_all('span')]

['Wed Aug 31, 2022',
 'Wed Aug 24, 2022',
 'Wed Aug 17, 2022',
 '3529',
 '3114',
 '822',
 '$100.00 - $120.00',
 '$100.00 - $118.00',
 '$105.00 - $118.00',
 '$85.00 - $99.00',
 '$90.00 - $99.00',
 '$95.00 - $104.00',
 '$88.00 - $94.00',
 '$88.00 - $96.00',
 '$85.00 - $98.00',
 '$74.00 - $87.00',
 '$70.00 - $87.00',
 '$70.00 - $84.00',
 '$120.00 - $155.00',
 '$120.00 - $155.00',
 '$120.00 - $155.00',
 '$115.00 - $155.00',
 '$115.00 - $147.00',
 '$120.00 - $143.00',
 '$125.00 - $160.00',
 '$125.00 - $160.00',
 '$120.00 - $150.00',
 '— - —',
 '— - —',
 '— - —',
 '— - —',
 '— - —',
 '— - —',
 '— - —',
 '— - —',
 '$1,800.00 - $1,950.00',
 '— - —',
 '— - —',
 '— - —',
 '— - —',
 '— - —',
 '— - —',
 '$215.00 - $226.75',
 '$170.00 - $203.00',
 '$218.00 - $224.25',
 '$180.00 - $198.50',
 '$194.00 - $209.00',
 '$160.00 - $208.50',
 '$225.00 - $241.50',
 '$195.00 - $219.50',
 '$225.00 - $235.50',
 '$200.00 - $217.00',
 '$195.00 - $209.00',
 '$180.00 - $205.00',
 '$230.00 - $253.00',
 '$210.00 - $2

In [5]:
okie_panokie = pd.read_html(vjv_ponoka_url)
okie_panokie

[   Click in table to get category history.             Last Auction  \
                               Week Ending:          Wed Sep 7, 2022   
 0                               Head Sold:                     1476   
 1                         Slaughter Cattle         Slaughter Cattle   
 2                             D1 - D2 Cows         $94.00 - $114.00   
 3                             D3 - D4 Cows          $80.00 - $93.00   
 4                       Good Holstein Cows          $83.00 - $88.00   
 5                     Medium Holstein Cows          $72.00 - $82.00   
 6                              Heiferettes        $120.00 - $155.00   
 7                            Bologna Bulls        $110.00 - $138.50   
 8                             Feeder Bulls        $120.00 - $150.00   
 9                       Replacement Cattle       Replacement Cattle   
 10                          Good Bred Cows                    — - —   
 11                         Older Bred Cows                    —

In [8]:
type(okie_panokie)

list

In [6]:
print(type(okie_panokie[0]))
print(len(okie_panokie))
print(len(okie_panokie[0]))

<class 'pandas.core.frame.DataFrame'>
1
36


In [7]:
ponoka_df = okie_panokie[0]
ponoka_df.info()
ponoka_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 7 columns):
 #   Column                                                   Non-Null Count  Dtype 
---  ------                                                   --------------  ----- 
 0   (Click in table to get category history., Week Ending:)  36 non-null     object
 1   (Last Auction, Wed Sep 7, 2022)                          36 non-null     object
 2   (Last Auction, Wed Sep 7, 2022.1)                        36 non-null     object
 3   (Previous Auction 1, Wed Aug 31, 2022)                   36 non-null     object
 4   (Previous Auction 1, Wed Aug 31, 2022.1)                 36 non-null     object
 5   (Previous Auction 2, Wed Aug 24, 2022)                   36 non-null     object
 6   (Previous Auction 2, Wed Aug 24, 2022.1)                 36 non-null     object
dtypes: object(7)
memory usage: 2.1+ KB


Click in table to get category history.             Last Auction  \
                              Week Ending:          Wed Sep 7, 2022   
0                               Head Sold:                     1476   
1                         Slaughter Cattle         Slaughter Cattle   
2                             D1 - D2 Cows         $94.00 - $114.00   
3                             D3 - D4 Cows          $80.00 - $93.00   
4                       Good Holstein Cows          $83.00 - $88.00   
5                     Medium Holstein Cows          $72.00 - $82.00   
6                              Heiferettes        $120.00 - $155.00   
7                            Bologna Bulls        $110.00 - $138.50   
8                             Feeder Bulls        $120.00 - $150.00   
9                       Replacement Cattle       Replacement Cattle   
10                          Good Bred Cows                    — - —   
11                         Older Bred Cows                    — - —   
12                       Good Bred Heifers                    — - —   
13              Cow / Calf Pairs (Younger)                    — - —   
14                Cow / Calf Pairs (Older)                    — - —   
15                 Stock and Feeder Cattle  Stock and Feeder Cattle   
16                                  Weight                   Steers   
17               1000 lbs Steers | Heifers        $214.00 - $220.00   
18                900 lbs Steers | Heifers        $210.00 - $236.50   
19                800 lbs Steers | Heifers        $235.00 - $253.00   
20                700 lbs Steers | Heifers        $235.00 - $257.00   
21                600 lbs Steers | Heifers        $240.00 - $267.00   
22                500 lbs Steers | Heifers        $245.00 - $276.00   
23                400 lbs Steers | Heifers        $250.00 - $284.00   
24                300 lbs Steers | Heifers        $250.00 - $305.00   
25                          Dairy & Calves           Dairy & Calves   
26                            Dairy Steers        $120.00 - $160.00   
27                Baby Calves (Dairy) $/Hd                    — - —   
28                 Baby Calves (Beef) $/Hd                    — - —   
29                                    Feed                     Feed   
30                       Hay: Square Bales                    — - —   
31                        Hay: Round Bales                    — - —   
32                     Straw: Square Bales                — - $2.50   
33                      Straw: Round Bales                    — - —   
34                 Greenfeed: Square Bales                    — - —   
35                  Greenfeed: Round Bales                    — - —   

                                  Previous Auction 1                           \
          Wed Sep 7, 2022.1         Wed Aug 31, 2022       Wed Aug 31, 2022.1   
0                      1476                     3529                     3529   
1          Slaughter Cattle         Slaughter Cattle         Slaughter Cattle   
2          $94.00 - $114.00        $100.00 - $120.00        $100.00 - $120.00   
3           $80.00 - $93.00          $85.00 - $99.00          $85.00 - $99.00   
4           $83.00 - $88.00          $88.00 - $94.00          $88.00 - $94.00   
5           $72.00 - $82.00          $74.00 - $87.00          $74.00 - $87.00   
6         $120.00 - $155.00        $120.00 - $155.00        $120.00 - $155.00   
7         $110.00 - $138.50        $115.00 - $155.00        $115.00 - $155.00   
8         $120.00 - $150.00        $125.00 - $160.00        $125.00 - $160.00   
9        Replacement Cattle       Replacement Cattle       Replacement Cattle   
10                    — - —                    — - —                    — - —   
11                    — - —                    — - —                    — - —   
12                    — - —                    — - —                    — - —   
13                    — - —                    — - —                    — - —   
14      

In [8]:
ponoka_df = ponoka_df.T
ponoka_df

0   \
Click in table to get category history. Week Ending:        Head Sold:   
Last Auction                            Wed Sep 7, 2022           1476   
                                        Wed Sep 7, 2022.1         1476   
Previous Auction 1                      Wed Aug 31, 2022          3529   
                                        Wed Aug 31, 2022.1        3529   
Previous Auction 2                      Wed Aug 24, 2022          3114   
                                        Wed Aug 24, 2022.1        3114   

                                                                          1   \
Click in table to get category history. Week Ending:        Slaughter Cattle   
Last Auction                            Wed Sep 7, 2022     Slaughter Cattle   
                                        Wed Sep 7, 2022.1   Slaughter Cattle   
Previous Auction 1                      Wed Aug 31, 2022    Slaughter Cattle   
                                        Wed Aug 31, 2022.1  Slaughter Cattle   
Previous Auction 2                      Wed Aug 24, 2022    Slaughter Cattle   
                                        Wed Aug 24, 2022.1  Slaughter Cattle   

                                                                           2   \
Click in table to get category history. Week Ending:             D1 - D2 Cows   
Last Auction                            Wed Sep 7, 2022      $94.00 - $114.00   
                                        Wed Sep 7, 2022.1    $94.00 - $114.00   
Previous Auction 1                      Wed Aug 31, 2022    $100.00 - $120.00   
                                        Wed Aug 31, 2022.1  $100.00 - $120.00   
Previous Auction 2                      Wed Aug 24, 2022    $100.00 - $118.00   
                                        Wed Aug 24, 2022.1  $100.00 - $118.00   

                                                                         3   \
Click in table to get category history. Week Ending:           D3 - D4 Cows   
Last Auction                            Wed Sep 7, 2022     $80.00 - $93.00   
                                        Wed Sep 7, 2022.1   $80.00 - $93.00   
Previous Auction 1                      Wed Aug 31, 2022    $85.00 - $99.00   
                                        Wed Aug 31, 2022.1  $85.00 - $99.00   
Previous Auction 2                      Wed Aug 24, 2022    $90.00 - $99.00   
                                        Wed Aug 24, 2022.1  $90.00 - $99.00   

                                                                            4   \
Click in table to get category history. Week Ending:        Good Holstein Cows   
Last Auction                            Wed Sep 7, 2022        $83.00 - $88.00   
                                        Wed Sep 7, 2022.1      $83.00 - $88.00   
Previous Auction 1                      Wed Aug 31, 2022       $88.00 - $94.00   
                                        Wed Aug 31, 2022.1     $88.00 - $94.00   
Previous Auction 2                      Wed Aug 24, 2022       $88.00 - $96.00   
                                        Wed Aug 24, 2022.1     $88.00 - $96.00   

                                                                              5   \
Click in table to get category history. Week Ending:        Medium Holstein Cows   
Last Auction                            Wed Sep 7, 2022          $72.00 - $82.00   
                                        Wed Sep 7, 2022.1        $72.00 - $82.00   
Previous Auction 1                      Wed Aug 31, 2022         $74.00 - $87.00   
                                        Wed Aug 31, 2022.1       $74.00 - $87.00   
Previous Auction 2                      Wed Aug 24, 2022         $70.00 - $87.00   
                                        Wed Aug 24, 2022.1       $70.00 - $87.00   

                                                                           6   \
Click in table to get category history. Week Ending:              Heiferettes   
Last Auction                            Wed Sep 7, 2022     

In [9]:
ponoka_df.iloc[0]

0                     Head Sold:
1               Slaughter Cattle
2                   D1 - D2 Cows
3                   D3 - D4 Cows
4             Good Holstein Cows
5           Medium Holstein Cows
6                    Heiferettes
7                  Bologna Bulls
8                   Feeder Bulls
9             Replacement Cattle
10                Good Bred Cows
11               Older Bred Cows
12             Good Bred Heifers
13    Cow / Calf Pairs (Younger)
14      Cow / Calf Pairs (Older)
15       Stock and Feeder Cattle
16                        Weight
17     1000 lbs Steers | Heifers
18      900 lbs Steers | Heifers
19      800 lbs Steers | Heifers
20      700 lbs Steers | Heifers
21      600 lbs Steers | Heifers
22      500 lbs Steers | Heifers
23      400 lbs Steers | Heifers
24      300 lbs Steers | Heifers
25                Dairy & Calves
26                  Dairy Steers
27      Baby Calves (Dairy) $/Hd
28       Baby Calves (Beef) $/Hd
29                          Feed
30        

In [10]:
type(ponoka_df.iloc[0])

pandas.core.series.Series

In [11]:
ponoka_df.columns = ponoka_df.iloc[0]
ponoka_df.columns = [col.lower()\
    .replace(' ', '_')\
        .replace('_/_', '_')\
            .replace('-', 'to')\
                .replace('_|_', '_')\
                    .replace('$', 'cad')\
                        .replace(':', '')\
                            .replace('(', '')\
                                .replace(')', '') for col in ponoka_df.columns]
ponoka_df.head()

head_sold  \
Click in table to get category history. Week Ending:        Head Sold:   
Last Auction                            Wed Sep 7, 2022           1476   
                                        Wed Sep 7, 2022.1         1476   
Previous Auction 1                      Wed Aug 31, 2022          3529   
                                        Wed Aug 31, 2022.1        3529   

                                                            slaughter_cattle  \
Click in table to get category history. Week Ending:        Slaughter Cattle   
Last Auction                            Wed Sep 7, 2022     Slaughter Cattle   
                                        Wed Sep 7, 2022.1   Slaughter Cattle   
Previous Auction 1                      Wed Aug 31, 2022    Slaughter Cattle   
                                        Wed Aug 31, 2022.1  Slaughter Cattle   

                                                                d1_to_d2_cows  \
Click in table to get category history. Week Ending:             D1 - D2 Cows   
Last Auction                            Wed Sep 7, 2022      $94.00 - $114.00   
                                        Wed Sep 7, 2022.1    $94.00 - $114.00   
Previous Auction 1                      Wed Aug 31, 2022    $100.00 - $120.00   
                                        Wed Aug 31, 2022.1  $100.00 - $120.00   

                                                              d3_to_d4_cows  \
Click in table to get category history. Week Ending:           D3 - D4 Cows   
Last Auction                            Wed Sep 7, 2022     $80.00 - $93.00   
                                        Wed Sep 7, 2022.1   $80.00 - $93.00   
Previous Auction 1                      Wed Aug 31, 2022    $85.00 - $99.00   
                                        Wed Aug 31, 2022.1  $85.00 - $99.00   

                                                            good_holstein_cows  \
Click in table to get category history. Week Ending:        Good Holstein Cows   
Last Auction                            Wed Sep 7, 2022        $83.00 - $88.00   
                                        Wed Sep 7, 2022.1      $83.00 - $88.00   
Previous Auction 1                      Wed Aug 31, 2022       $88.00 - $94.00   
                                        Wed Aug 31, 2022.1     $88.00 - $94.00   

                                                            medium_holstein_cows  \
Click in table to get category history. Week Ending:        Medium Holstein Cows   
Last Auction                            Wed Sep 7, 2022          $72.00 - $82.00   
                                        Wed Sep 7, 2022.1        $72.00 - $82.00   
Previous Auction 1                      Wed Aug 31, 2022         $74.00 - $87.00   
                                        Wed Aug 31, 2022.1       $74.00 - $87.00   

                                                                  heiferettes  \
Click in table to get category history. Week Ending:              Heiferettes   
Last Auction                            Wed Sep 7, 2022     $120.00 - $155.00   
                                        Wed Sep 7, 2022.1   $120.00 - $155.00   
Previous Auction 1                      Wed Aug 31, 2022    $120.00 - $155.00   
                                        Wed Aug 31, 2022.1  $120.00 - $155.00   

                                                                bologna_bulls  \
Click in table to get category history. Week Ending:            Bologna Bulls   
Last Auction                            Wed Sep 7, 2022     $110.00 - $138.50   
                                        Wed Sep 7, 2022.1   $110.00 - $138.50   
Previous Auction 1                      Wed Aug 31, 2022    $115.00 - $155.00   
                                        Wed Aug 31, 2022.1  $115.00 - $155.00   

                                                                 feeder_bulls  \
Click in table to get category history. Week Ending:             Feeder Bulls   
Last Auction                           

In [12]:
ponoka_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 7 entries, ('Click in table to get category history.', 'Week\xa0Ending:') to ('Previous Auction 2', 'Wed Aug 24, 2022.1')
Data columns (total 36 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   head_sold                 7 non-null      object
 1   slaughter_cattle          7 non-null      object
 2   d1_to_d2_cows             7 non-null      object
 3   d3_to_d4_cows             7 non-null      object
 4   good_holstein_cows        7 non-null      object
 5   medium_holstein_cows      7 non-null      object
 6   heiferettes               7 non-null      object
 7   bologna_bulls             7 non-null      object
 8   feeder_bulls              7 non-null      object
 9   replacement_cattle        7 non-null      object
 10  good_bred_cows            7 non-null      object
 11  older_bred_cows           7 non-null      object
 12  good_bred_heifers         7 non-nu

In [14]:
ponoka_df = ponoka_df.iloc[1:]
ponoka_df.info()
ponoka_df.head()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5 entries, ('Last Auction', 'Wed Sep 7, 2022.1') to ('Previous Auction 2', 'Wed Aug 24, 2022.1')
Data columns (total 36 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   head_sold                 5 non-null      object
 1   slaughter_cattle          5 non-null      object
 2   d1_to_d2_cows             5 non-null      object
 3   d3_to_d4_cows             5 non-null      object
 4   good_holstein_cows        5 non-null      object
 5   medium_holstein_cows      5 non-null      object
 6   heiferettes               5 non-null      object
 7   bologna_bulls             5 non-null      object
 8   feeder_bulls              5 non-null      object
 9   replacement_cattle        5 non-null      object
 10  good_bred_cows            5 non-null      object
 11  older_bred_cows           5 non-null      object
 12  good_bred_heifers         5 non-null      object
 13  cow_c

head_sold  slaughter_cattle  \
Last Auction       Wed Sep 7, 2022.1       1476  Slaughter Cattle   
Previous Auction 1 Wed Aug 31, 2022        3529  Slaughter Cattle   
                   Wed Aug 31, 2022.1      3529  Slaughter Cattle   
Previous Auction 2 Wed Aug 24, 2022        3114  Slaughter Cattle   
                   Wed Aug 24, 2022.1      3114  Slaughter Cattle   

                                           d1_to_d2_cows    d3_to_d4_cows  \
Last Auction       Wed Sep 7, 2022.1    $94.00 - $114.00  $80.00 - $93.00   
Previous Auction 1 Wed Aug 31, 2022    $100.00 - $120.00  $85.00 - $99.00   
                   Wed Aug 31, 2022.1  $100.00 - $120.00  $85.00 - $99.00   
Previous Auction 2 Wed Aug 24, 2022    $100.00 - $118.00  $90.00 - $99.00   
                   Wed Aug 24, 2022.1  $100.00 - $118.00  $90.00 - $99.00   

                                      good_holstein_cows medium_holstein_cows  \
Last Auction       Wed Sep 7, 2022.1     $83.00 - $88.00      $72.00 - $82.00   
Previous Auction 1 Wed Aug 31, 2022      $88.00 - $94.00      $74.00 - $87.00   
                   Wed Aug 31, 2022.1    $88.00 - $94.00      $74.00 - $87.00   
Previous Auction 2 Wed Aug 24, 2022      $88.00 - $96.00      $70.00 - $87.00   
                   Wed Aug 24, 2022.1    $88.00 - $96.00      $70.00 - $87.00   

                                             heiferettes      bologna_bulls  \
Last Auction       Wed Sep 7, 2022.1   $120.00 - $155.00  $110.00 - $138.50   
Previous Auction 1 Wed Aug 31, 2022    $120.00 - $155.00  $115.00 - $155.00   
                   Wed Aug 31, 2022.1  $120.00 - $155.00  $115.00 - $155.00   
Previous Auction 2 Wed Aug 24, 2022    $120.00 - $155.00  $115.00 - $147.00   
                   Wed Aug 24, 2022.1  $120.00 - $155.00  $115.00 - $147.00   

                                            feeder_bulls  replacement_cattle  \
Last Auction       Wed Sep 7, 2022.1   $120.00 - $150.00  Replacement Cattle   
Previous Auction 1 Wed Aug 31, 2022    $125.00 - $160.00  Replacement Cattle   
                   Wed Aug 31, 2022.1  $125.00 - $160.00  Replacement Cattle   
Previous Auction 2 Wed Aug 24, 2022    $125.00 - $160.00  Replacement Cattle   
                   Wed Aug 24, 2022.1  $125.00 - $160.00  Replacement Cattle   

                                      good_bred_cows older_bred_cows  \
Last Auction       Wed Sep 7, 2022.1           — - —           — - —   
Previous Auction 1 Wed Aug 31, 2022            — - —           — - —   
                   Wed Aug 31, 2022.1          — - —           — - —   
Previous Auction 2 Wed Aug 24, 2022            — - —           — - —   
                   Wed Aug 24, 2022.1          — - —           — - —   

                                      good_bred_heifers  \
Last Auction       Wed Sep 7, 2022.1              — - —   
Previous Auction 1 Wed Aug 31, 2022               — - —   
                   Wed Aug 31, 2022.1             — - —   
Previous Auction 2 Wed Aug 24, 2022               — - —   
                   Wed Aug 24, 2022.1             — - —   

                                      cow_calf_pairs_younger  \
Last Auction       Wed Sep 7, 2022.1                   — - —   
Previous Auction 1 Wed Aug 31, 2022                    — - —   
                   Wed Aug 31, 2022.1                  — - —   
Previous Auction 2 Wed Aug 24, 2022                    — - —   
                   Wed Aug 24, 2022.1                  — - —   

                                      cow_calf_pairs_older  \
Last Auction       Wed Sep 7, 2022.1                 — - —   
Previous Auction 1 Wed Aug 31, 2022                  — - —   
                   Wed Aug 31, 2022.1                — - —   
Previous Auction 2 Wed Aug 24, 2022                  — - —   
                   Wed Aug 24, 2022.1                — - —   

                                       stock_and_feeder_cattle   weight  \
Last Auction       Wed Sep 7, 2022.1   Stock and Feeder Cattle  Heifers   
Previous Auction 1 We

In [16]:
print(ponoka_df.index)
type(ponoka_df.index)

MultiIndex([(      'Last Auction',  'Wed Sep 7, 2022.1'),
            ('Previous Auction 1',   'Wed Aug 31, 2022'),
            ('Previous Auction 1', 'Wed Aug 31, 2022.1'),
            ('Previous Auction 2',   'Wed Aug 24, 2022'),
            ('Previous Auction 2', 'Wed Aug 24, 2022.1')],
           )


pandas.core.indexes.multi.MultiIndex

In [25]:
ponoka_df.index.get_level_values(1)

Index(['Wed Sep 7, 2022.1', 'Wed Aug 31, 2022', 'Wed Aug 31, 2022.1',
       'Wed Aug 24, 2022', 'Wed Aug 24, 2022.1'],
      dtype='object')